In [9]:
!pip install pyro-ppl

     |████████████████████████████████| 522kB 2.0MB/s 


In [0]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib.lines import Line2D
import seaborn as sns

import torch
from torch import nn
from torch.distributions import constraints
import functools

import pyro
import pyro.distributions as dist
from pyro.infer import SVI, JitTraceEnum_ELBO, TraceEnum_ELBO
from pyro.contrib.autoguide import AutoDiagonalNormal, AutoMultivariateNormal, AutoGuideList, AutoDelta
from pyro.optim import ClippedAdam

import time

# fix MCMCndom generator seed (for reproducibility of results)
np.random.seed(42)

# matplotlib style options
plt.style.use('ggplot')
%matplotlib inline
plt.rcParams['figure.figsize'] = (16, 10)

In [0]:

def model(data=None, T, obs=None):
    """ Define priors over beta1, beta2, tau, noises, sigma, z_prev1 and z_prev2 (keep the shapes in mind)
    # Your code here
    """

    psi = pyro.sample(name="psi", fn=dist.HalfCauchy(scale=3.))
    delta = pyro.sample(name="delta", fn=dist.HalfCauchy(scale=3.))
    sigma = pyro.sample(name="sigma", fn=dist.HalfCauchy(scale=3.))

    alpha_prev = pyro.sample(name="alpha_prev", fn=dist.Normal(loc=torch.zeros(100), scale=delta))
    beta_prev = pyro.sample(name="beta_prev", fn=dist.Normal(loc=toruch.zeros(25), scale=sigma))
    
    ALPHA = [alpha_prev] 
    BETA =  [beta_prev]

    for t in range(1, T):
        alpha_t = pyro.sample(name="alpha_prev", fn=dist.Normal(loc=alpha_prev, scale=delta))
        beta_t = pyro.sample(name="beta_prev", fn=dist.Normal(loc=beta_prev, scale=sigma))
        
        ALPHA.append(alpha_t)
        BETA.append(beta_t)

        alpha_prev = alpha_t
        beta_prev = beta_t 

        """ Make a plate of size num_topics with name "topics" and define a variable "topic_words".
          This represents the phi above. Use the equivalent of a uniform distribution for it  """
        # with pyro.plate("topics", num_topics):
                # topic_words
                # theta = pyro.sample("theta", dist.Normal(loc = alpha_t, scale = psi))

            """ Make two (nested) plates  in here. One over documents and one over words
                Documents, called "documents":
                The plate over the documents should hold a variable "doc_topics" representing the theta above.
                    Use the equivalent of a uniform distribution for it.
                
                Words, called "words":
                The plate over words, should have a topic assignment for each word (z_{i,j} above) which 
                    should be enumerated.
                The second variable should be the words themselves which should be drawn from the "topic_words"
                    using the assigned z_{i,j} and the observed data.

            """
        with pyro.plate("documents", num_docs) as ind:
            if data is not None:
                with pyro.util.ignore_jit_warnings():
                    assert data.shape == (num_words_per_doc, num_docs)
                data = data[:, ind]
            
            theta = pyro.sample("theta", dist.Normal(loc = alpha_t, scale = psi))

            # doc_topics = pyro.sample("doc_topics", dist.Categorical(torch.ones(num_topics)/ num_topics))
            
            with pyro.plate("words", num_words_per_doc):
                # The word_topics variable is marginalized out during inference,
                # achieved by specifying infer={"enumerate": "parallel"} and using
                # TraceEnum_ELBO for inference. Thus we can ignore this variable in
                # the guide.


                z = pyro.sample("z", dist.Categorical(logits = theta))
                w = pyro.sample("w", dist.Categorical(logits = beta_t[z]]), obs=data)
    

Normal(loc: torch.Size([100]), scale: torch.Size([100]))